### Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

In [2]:
ROOT_PATH = '/Users/nastya/dev/time_series_prediction/'

### Base algorithm errors

In [50]:
errors_table = pd.DataFrame(columns=[('beta, %', ''), ('strategy / UP', ''), \
    ('NP', ''), ('Self-healing', ''), \
    ('h=1', 'NP, %'), ('h=1', 'MAPE, %'), ('h=1', 'RMSE'),\
    ('h=10', 'NP, %'), ('h=10', 'MAPE, %'), ('h=10', 'RMSE'),
    ('h=50', 'NP, %'), ('h=50', 'MAPE, %'), ('h=50', 'RMSE'),
    ('h=100', 'NP, %'), ('h=100', 'MAPE, %'), ('h=100', 'RMSE')])

In [51]:
errors_table.columns = pd.MultiIndex.from_tuples(errors_table.columns)

In [4]:
errors_table_template = {
    ('beta, %', '') : 0, 
    ('strategy / UP', '') : 0, 
    ('NP', '') : 0, 
    ('Self-healing', '') : 0,
    ('h=1', 'NP, %') : 0, 
    ('h=1', 'MAPE, %') : 0,
    ('h=1', 'RMSE') : 0,
    ('h=10', 'NP, %') : 0, 
    ('h=10', 'MAPE, %') : 0, 
    ('h=10', 'RMSE') : 0,
    ('h=50', 'NP, %') : 0,
    ('h=50', 'MAPE, %') : 0, 
    ('h=50', 'RMSE') : 0,
    ('h=100', 'NP, %') : 0, 
    ('h=100', 'MAPE, %') : 0, 
    ('h=100', 'RMSE') : 0
}

In [21]:
def non_pred(Y_pred):
    h = len(Y_pred)
    return np.count_nonzero(Y_pred == 'N') / h * 100

def rmse(Y_true, Y_pred):
    predictable = np.argwhere(Y_pred != 'N').reshape(1, -1)[0]
    if len(predictable) == 0:
        return 0
    return np.sqrt(mean_squared_error(np.take(Y_true, predictable), \
        np.take(Y_pred, predictable)))

def mape(Y_true, Y_pred):
    predictable = np.argwhere(Y_pred != 'N').reshape(1, -1)[0]
    if len(predictable) == 0:
        return 0
    return mean_absolute_percentage_error(np.take(Y_true, predictable), \
        np.take(Y_pred, predictable))

In [7]:
# Y = pd.read_csv(ROOT_PATH + 'data/lorenz.txt', sep="\n", header=None)
with open(ROOT_PATH + 'data/lorenz.dat', 'rb') as f:
    Y = pickle.load(f)
n_train = 10_000
n_test = 1_000 + 300 # to get test set of 1000
n_passed = 3_000
n_valid = 2_000

# min_Y = min(Y)
# max_Y = max(Y)
# Y = [(y - min_Y) / (max_Y - min_Y) for y in Y]

Y1 = np.array(Y[n_passed:n_passed + n_train]).reshape(-1)
Y2 = np.array(Y[n_passed + n_train:n_passed + n_train + n_test]).reshape(-1)
Y3 = np.array(Y[n_passed + n_train + n_test:n_passed + n_train + n_test + n_valid]).reshape(-1)
Y_true = Y2[131:]

In [52]:
# putting in np_method experiment
def add_row(errors_table, pm_filepath, beta, strategy_up, np_method, self_healing):
    with open(pm_filepath, 'rb') as f:
        pm = pickle.load(f)
    n_iterations = len(pm[0])

    if errors_table[(errors_table[('beta, %', '')] == beta) &\
                    (errors_table[('strategy / UP', '')] == strategy_up) &\
                    (errors_table[('NP', '')] == np_method) &\
                    (errors_table[('Self-healing', '')] == self_healing)].shape[0] > 0:
        return errors_table

    errors_table = errors_table.append({
        ('beta, %', '') : beta, 
        ('strategy / UP', '') : strategy_up, 
        ('NP', '') : np_method, 
        ('Self-healing', '') : self_healing,
        ('h=1', 'NP, %') : non_pred(pm[1]), 
        ('h=1', 'MAPE, %') : mape(Y_true[:n_iterations], pm[1]) * 100,
        ('h=1', 'RMSE') : rmse(Y_true[:n_iterations], pm[1]),
        ('h=10', 'NP, %') : non_pred(pm[10]), 
        ('h=10', 'MAPE, %') : mape(Y_true[:n_iterations], pm[10]) * 100, 
        ('h=10', 'RMSE') : rmse(Y_true[:n_iterations], pm[10]),
        ('h=50', 'NP, %') : non_pred(pm[50]),
        ('h=50', 'MAPE, %') : mape(Y_true[:n_iterations], pm[50]) * 100, 
        ('h=50', 'RMSE') : rmse(Y_true[:n_iterations], pm[50]),
        ('h=100', 'NP, %') : non_pred(pm[100]), 
        ('h=100', 'MAPE, %') : mape(Y_true[:n_iterations], pm[100]) * 100, 
        ('h=100', 'RMSE') : rmse(Y_true[:n_iterations], pm[100])
        }, ignore_index=True)
    return errors_table

In [53]:
errors_table = add_row(errors_table, ROOT_PATH + 'results/predictions/base_b20/pm_fp.dat', 20, 's / db', 'fp', 'n')
errors_table = add_row(errors_table, ROOT_PATH + 'results/predictions/base_b20/pm_rd.dat', 20, 's / db', 'rd', 'n')
errors_table = add_row(errors_table, ROOT_PATH + 'results/predictions/base_b20/pm_ls.dat', 20, 's / db', 'ls', 'n')
errors_table = add_row(errors_table, ROOT_PATH + 'results/predictions/base_b20/pm_rg.dat', 20, 's / db', 'rg', 'n')
errors_table = add_row(errors_table, ROOT_PATH + 'results/predictions/base_b20/pm_rw.dat', 20, 's / db', 'rw', 'n')
errors_table = add_row(errors_table, ROOT_PATH + 'results/predictions/base_b20/pm_cl_40_oc.dat', 20, 's / db', 'cl_40_oc', 'n')

1000
1000
1000
1000
1000
1000


In [54]:
errors_table

beta, % strategy / UP        NP Self-healing   h=1                      \
                                               NP, %   MAPE, %      RMSE   
0      20        s / db        fp            n  31.9  2.687178  0.024682   
1      20        s / db        rd            n  32.4  2.666047  0.024084   
2      20        s / db        ls            n  83.6  1.048291  0.008025   
3      20        s / db        rg            n  31.9  2.687178  0.024682   
4      20        s / db        rw            n  32.9  2.670949  0.024152   
5      20        s / db  cl_40_oc            n  42.1  2.552072  0.025064   

   h=10                       h=50                       h=100              \
  NP, %   MAPE, %      RMSE  NP, %    MAPE, %      RMSE  NP, %     MAPE, %   
0  41.1  6.137749  0.048368   61.9  70.139778  0.299465   73.7   68.263074   
1  45.0  5.946253  0.048923   68.7  57.711004  0.278128   80.0   74.748519   
2  83.3  2.720782  0.017645  100.0   0.000000  0.000000  100.0    0.000000   
3  45.5  6.115934  0.049349   75.4  73.090579  0.290742   91.4   98.252740   
4  52.2  5.636543  0.046102   83.7  59.879716  0.241407   96.4  124.601875   
5  54.2  5.724651  0.049436   80.9  65.726679  0.282946   87.3   79.023686   

             
       RMSE  
0  0.318410  
1  0.309771  
2  0.000000  
3  0.289827  
4  0.344821  
5  0.340574

In [45]:
errors_table.to_csv(ROOT_PATH + 'results/1_errors_table.csv')

### Healing 5 points thrown metrics

In [9]:
healing_params = {
    'healing_up_method' : 'db', 
    'weighted_up' : True,  
    # {'double_clustering', 'weighred_average', 'pointwise_weights'}
    'weight_method' : 'pointwise_weights', 
    'clear_noise' : True,
    'factor' : 0.4,
    'alg_type' : 's',
    'np_method' : 'rd',
    'mc_method' : 'db',
    'beta' : 0.2,
    'fixed_points' : True,
}

In [10]:
healing_5_thrown_errors = pd.DataFrame(columns=list(healing_params.keys()) + \
    ['RMSE', 'MAPE', 'non-predictable', 'n_iterations'])

In [4]:
healing_5_thrown_errors = pd.read_csv(ROOT_PATH + 'results/graphs/04.01-07/errors.csv', index_col=0)

In [5]:
healing_5_thrown_errors.columns

Index(['healing_up_method', 'weighted_up', 'weight_method', 'clear_noise',
       'factor', 'alg_type', 'np_method', 'mc_method', 'beta', 'fixed_points',
       'RMSE', 'MAPE', 'non-predictable', 'n_iterations', 'healing_match_eps'],
      dtype='object')

In [60]:
filename = 'results/predictions/five_thrown_experiments/logs_healing_5_points_thrown_rg_weightedv2_check.dat'
with open(ROOT_PATH + filename, 'rb') as f:
    up_logs = pickle.load(f)

params_and_errors = {
    'healing_up_method' : 'db', 
    'weighted_up' : True,  
    # {'double_clustering', 'weighred_average', 'pointwise_weights'}
    'weight_method' : 'weighted_average', 
    'clear_noise' : False,
    'factor' : None,
    'alg_type' : 's',
    'np_method' : 'rd',
    'mc_method' : 'db',
    'beta' : 0.2,
    'fixed_points' : False,
    'RMSE' : rmse(Y2[:100], up_logs[-1]),
    'MAPE' : mape(Y2[:100], up_logs[-1]),
    'non-predictable' : non_pred(up_logs[-1]),
    'n_iterations' : len(up_logs),
    'healing_match_eps' : 0.01
}
# params_and_errors

healing_5_thrown_errors = healing_5_thrown_errors.append(params_and_errors, \
    ignore_index=True)

In [59]:
healing_5_thrown_errors.to_csv(ROOT_PATH + 'results/graphs/04.01-07/errors.csv')

In [81]:
with open(ROOT_PATH + 'code/logs_real_prediction_tp_rd_wi_basealg_jdistfactor.dat', 'rb') as f:
    up_logs = pickle.load(f)

res = {
    'alg': 'trajectory prediction + self-healing, rd_wi_basealg_jdistfactor',
    'RMSE' : rmse(Y2[:100], up_logs[-1]),
    'MAPE' : mape(Y2[:100], up_logs[-1]),
    'non-predictable' : non_pred(up_logs[-1]),
}

In [82]:
res

{'alg': 'trajectory prediction + self-healing, rd_wi_basealg_jdistfactor',
 'RMSE': 0.3148395265409664,
 'MAPE': 0.5829637404047658,
 'non-predictable': 1.0}

### One sample t-test

In [36]:
from scipy.stats import ttest_1samp

In [37]:
def get_p_value_of_ttest(pm1, pm2, hs1, hs2):
    n_iterations = len(pm1[0])
    n_test_passed = 131
    rmse1 = [rmse(Y2[n_test_passed:n_test_passed + n_iterations], pm1[i]) for i in hs1]
    rmse2 = [rmse(Y2[n_test_passed:n_test_passed + n_iterations], pm2[i]) for i in hs2]

    statictics, pvalue = ttest_1samp(rmse1, rmse2)
    return statictics, pvalue


In [38]:
with open(ROOT_PATH + 'results/predictions/05.06-13/find best base algorithm/n_iterations 250/pm_cs_0.1_n_1.dat', 'rb') as f:
    pm1 = pickle.load(f)

with open(ROOT_PATH + 'results/predictions/05.06-13/find best base algorithm/n_iterations 250/pm_cs_0.1_n_5.dat', 'rb') as f:
    pm2 = pickle.load(f)

hs = list(range(1, 101))
print(get_p_value_of_ttest(pm1, pm2, hs, hs))

(array([ 18.5389294 ,  18.49610193,  18.36150667,  18.01089493,
        17.80016024,  17.75922264,  17.76365273,  17.59432149,
        16.28460815,  16.5263683 ,  15.84480421,  16.86968661,
        15.83181959,  14.76849503,  16.3464289 ,  16.81235321,
        14.65589928,  15.28381619,  14.60020674,   9.96744838,
         8.28962163,   6.45679351,   4.49657343,   4.36619069,
         5.46120608,   3.5627119 ,   1.27566976,  -1.55601174,
        -4.6389129 ,  -6.39346374,  -7.4044148 ,  -7.88523417,
        -7.18186091,  -6.41950657,  -7.67099686,  -9.21967475,
       -11.29596402, -11.83509185, -13.11936526, -12.78414277,
        -9.14894767,  -5.21776963,  -6.69645507,  -5.73770678,
        -7.8553568 , -10.73852954, -11.56355807, -10.81504446,
        -7.70122981,  -5.44797377,  -3.76184209,  -1.78483951,
        -1.82601283,  -5.00667059,  -5.2144323 ,  -1.17211805,
         4.62713069,   5.35971961,   1.17073347,  -3.00678312,
        -8.8340582 ,  -9.7200041 , -10.6863969 ,  -5.7

In [34]:
from scipy.stats.distributions import chi2
n_iterations = len(pm1[0])
n_test_passed = 131
rmse1 = [rmse(Y2[n_test_passed:n_test_passed + n_iterations], pm1[i]) for i in hs]
rmse2 = [rmse(Y2[n_test_passed:n_test_passed + n_iterations], pm2[i]) for i in hs]


def likelihood_ratio(llmin, llmax):
    return(2*(llmax-llmin))


LR = likelihood_ratio(np.array(rmse1), np.array(rmse2))
p = chi2.sf(LR, 1)
print(p)

[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.92529571
 1.         1.         1.         0.87571395 1.         0.87801211
 0.89508666 1.         1.         1.         0.81083131 0.7681759
 0.82315458 1.         1.         1.         1.         0.84582513
 0.74187217 0.7329865  0.83103305 1.         1.         1.
 0.82706927 0.79855496 0.76638037 0.74034181 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.87156591 1.         0.74193    0.75361654 0.76585988
 0.70527294 0.66159531 0.65680363 0.79805643 0.80371804 0.75558377
 0.8318882  1.         0.84766569 0.95675549 0.77000257 0.83923799
 0.72851321 0.73012323 0.83358297 0.89036872 0.79537394 0.74828289
 0.82307597 0.92811665 0.89839018 1.         0.74711314 0.6918662
 0.74146065 0.77611698 0.80365692 0.92304216 0.66016829 0.77106963
 0.72566718 0.70907739 0.69307186 0.72248493 0.8944564  1.
 1.         1.         1.         1. 